In [1]:
import os
import time
import cv2
import base64
import numpy as np
from clickhouse_driver import Client
from deepface import DeepFace
from datetime import datetime

In [9]:
# Fungsi untuk menangkap wajah dari kamera
def capture_face():
    # Buat folder untuk menyimpan foto
    output_directory = './temporary_photo'
    os.makedirs(output_directory, exist_ok=True)

    # Inisialisasi objek OpenCV untuk kamera (0 adalah indeks kamera, bisa disesuaikan)
    cap = cv2.VideoCapture(0)

    # Variabel untuk menghitung jumlah foto yang sudah diambil
    photo_count = 0

    # Loop untuk membaca frame dari kamera
    while True:
        # Membaca frame
        ret, frame = cap.read()

        # Menampilkan frame
        cv2.imshow('Capture Face', frame)

        # Menunggu tombol 'q' untuk mengambil foto
        key = cv2.waitKey(1)
        if key == ord('q'):
            # Increment jumlah foto yang sudah diambil
            photo_count += 1

            # Menyusun path dan nama file untuk foto
            photo_name = f'foto_{photo_count}.jpg'
            photo_path = os.path.join(output_directory, photo_name)

            # Menyimpan foto
            cv2.imwrite(photo_path, frame)
            print(f"Photo {photo_count} saved as {photo_name} at {photo_path}")

            # Jika sudah mengambil 20 foto, keluar dari loop
            if photo_count == 20:
                break

    # Menghentikan kamera dan menutup jendela
    
    cap.release()
    cv2.destroyAllWindows()

def image_to_vector(img_path):
    try:
        # Gunakan DeepFace untuk mendapatkan vektor representasi dan base64
        result = DeepFace.represent(img_path, 
                                    model_name = "Facenet512",
                                    enforce_detection = False,
                                    detector_backend = "fastmtcnn")
        
        # Ambil nilai dari kunci pertama
        vector_representation = result[0]['embedding']

        # Ubah img_path ke base64
        with open(img_path, "rb") as image_file:
            base64_encoded = base64.b64encode(image_file.read()).decode('utf-8')

        return vector_representation, base64_encoded
    except Exception as e:
        print("Error:", str(e))
        return None, None

# Fungsi untuk menyimpan data karyawan ke tabel ClickHouse
def save_employee_data(employee_id, employee_name, vector_representation, base64_encoded):
    try:
        # Inisialisasi koneksi ClickHouse
        client = Client(host='localhost', port=9000, user='default', database='default')
        
        # Query untuk menyimpan data ke tabel karyawan
        query = f"INSERT INTO employee (employee_id, employee_name, embedding, image_base64) VALUES ({employee_id}, '{employee_name}', {vector_representation}, '{base64_encoded}');"

        # Eksekusi query
        client.execute(query)
    except Exception as e:
        print(f"Error: {e}")

def insert_ke_database():
    # Direktori tempat Anda menyimpan 20 foto
    input_directory = './temporary_photo'
    employee_name = input("Masukkan nama karyawan: ")
    employee_id = int(input("Masukkan ID karyawan: "))    

    capture_face()

    # Mengecek apakah direktori ada dan tidak kosong
    # if os.path.exists(input_directory) and os.listdir(input_directory):
    # Iterasi setiap file dalam direktori
    for filename in os.listdir(input_directory):
        if filename.endswith(".jpg"):
            # Path lengkap ke file
            img_path = os.path.join(input_directory, filename)

            # Menggunakan fungsi image_to_vector untuk mendapatkan vektor dan base64
            vector_representation, base64_encoded = image_to_vector(img_path)

            # Menggunakan fungsi save_employee_data untuk menyimpan data ke ClickHouse
            if vector_representation is not None and base64_encoded is not None:
                save_employee_data(employee_id, employee_name, vector_representation, base64_encoded)
                print(f"{employee_name} - {filename} berhasil dimasukkan ke database.")
    #else:
     #   print("Direktori kosong atau tidak ditemukan.")

insert_ke_database()

Masukkan nama karyawan:  Awa
Masukkan ID karyawan:  54321


Photo 1 saved as foto_1.jpg at ./temporary_photo\foto_1.jpg
Photo 2 saved as foto_2.jpg at ./temporary_photo\foto_2.jpg
Photo 3 saved as foto_3.jpg at ./temporary_photo\foto_3.jpg
Photo 4 saved as foto_4.jpg at ./temporary_photo\foto_4.jpg
Photo 5 saved as foto_5.jpg at ./temporary_photo\foto_5.jpg
Photo 6 saved as foto_6.jpg at ./temporary_photo\foto_6.jpg
Photo 7 saved as foto_7.jpg at ./temporary_photo\foto_7.jpg
Photo 8 saved as foto_8.jpg at ./temporary_photo\foto_8.jpg
Photo 9 saved as foto_9.jpg at ./temporary_photo\foto_9.jpg
Photo 10 saved as foto_10.jpg at ./temporary_photo\foto_10.jpg
Photo 11 saved as foto_11.jpg at ./temporary_photo\foto_11.jpg
Photo 12 saved as foto_12.jpg at ./temporary_photo\foto_12.jpg
Photo 13 saved as foto_13.jpg at ./temporary_photo\foto_13.jpg
Photo 14 saved as foto_14.jpg at ./temporary_photo\foto_14.jpg
Photo 15 saved as foto_15.jpg at ./temporary_photo\foto_15.jpg
Photo 16 saved as foto_16.jpg at ./temporary_photo\foto_16.jpg
Photo 17 saved as f

-------------------------------------------------------------------------------------------------------------------------------

In [2]:
c = Client(host='localhost', port=9000, user='default', database='default')
c.query_dataframe('select * from employee')

,employee_id,employee_name,embedding,image_base64
0,1234,Mas Uje,"[0.16301849484443665, -0.06630285829305649, -0...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
1,1234567,Asad,"[-0.7817592620849609, 0.765438437461853, -1.63...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
2,1234567,Asad,"[0.44773563742637634, 1.4925273656845093, -0.7...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
3,1234567,Asad,"[-0.022137485444545746, 1.2406355142593384, -1...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
4,1234567,Asad,"[0.22292783856391907, 0.6514220833778381, -1.5...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
...,...,...,...,...
95,1234,Mas Uje,"[0.3746837079524994, 2.2652294635772705, -1.04...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
96,1234,Mas Uje,"[0.785784900188446, 2.565462827682495, -1.1502...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
97,1234,Mas Uje,"[-0.09075675159692764, -0.09314879775047302, -...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
98,1234,Mas Uje,"[-0.18084916472434998, 2.173614978790283, -1.3...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...


In [4]:
today_date = datetime.now().strftime('%Y-%m-%d')

# Query SQL untuk memeriksa kehadiran
query = f"SELECT * FROM kehadiran WHERE employee_id = 121 AND employee_name = 'syahel' AND waktu_kehadiran >= '{today_date}'"

# Eksekusi query dengan menggunakan variabel client
hadir = client.execute(query)

In [175]:
hadir

[(121, 'syahel', datetime.datetime(2024, 2, 22, 10, 34, 34))]

In [178]:
datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2024-02-22 12:31:42'

In [31]:
result1 = DeepFace.represent('azka_tes.jpg', 
                            model_name = "Facenet512",
                            enforce_detection = False,
                            detector_backend = "fastmtcnn")
        
#Ambil nilai dari kunci pertama
vector_representation1 = result1[0]['embedding']

In [40]:
#print(vector_representation1)
result1[0]['facial_area']

{'x': 477, 'y': 518, 'w': 150, 'h': 189}

--------------------------------------------------------------------------------------------------------------

In [10]:
def get_face_representation(frame):
    # Save the frame as an image file
    image_path = 'temp_frame.jpg'
    cv2.imwrite(image_path, frame)

    # detected_face = DeepFace.detectFace(image_path=image_path, detector_backend='opencv')
    representation = DeepFace.represent(img_path=image_path, model_name='Facenet512', enforce_detection=False, detector_backend='fastmtcnn')
    face_representation = representation[0]['embedding']
    facial_area = list(representation[0]['facial_area'].values())
    base64_representation = base64.b64encode(cv2.imencode('.jpg', cv2.imread(image_path))[1]).decode()

    # Remove the temporary image file
    os.remove(image_path)

    return face_representation, base64_representation, facial_area

def record_attendance(employee_id, employee_name):
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    client.execute(
        f'INSERT INTO kehadiran (employee_id, employee_name, waktu_kehadiran) VALUES ({employee_id}, \'{employee_name}\', \'{current_time}\')'
    )

def update_or_insert_employee(employee_id, employee_name, embedding, base64_representation):
    client.execute(
        f'INSERT INTO employee (employee_id, employee_name, embedding, image_base64) '
        f'VALUES ({employee_id}, \'{employee_name}\', {embedding}, \'{base64_representation}\') '
    )

def display_attendance_info(frame, employee_id, employee_name, cosine_similarity, facial_area):
    # Gambar bounding box untuk wajah
    x, y, w, h = facial_area[0], facial_area[1], facial_area[2], facial_area[3]
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Tambahkan teks nama di bawah bounding box
    cv2.putText(frame, employee_name, (x, y + h + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Tambahkan teks similarity di bawah nama
    cv2.putText(frame, f'Similarity: {format(cosine_similarity, ".2%")}', (x, y + h + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Tampilkan frame di jendela
    cv2.imshow('Capture Face', frame)

def cek_kehadiran(employee_id, employee_name):
    # Ambil tanggal hari ini
    today_date = datetime.now().strftime('%Y-%m-%d')

    # Query SQL untuk memeriksa kehadiran
    query = f"SELECT * FROM kehadiran WHERE employee_id = '{employee_id}' AND employee_name = '{employee_name}' AND waktu_kehadiran >= '{today_date}'"

    # Eksekusi query dengan menggunakan variabel client
    hadir = client.execute(query)

    return hadir # if len(hadir) > 0 maka sudah hadir

def compare_faces(frame, embedding, facial_area):
    query_result = client.query_dataframe(
        f'SELECT employee_id, employee_name, 1 - cosineDistance({embedding}, embedding) AS cosineSimilarity, image_base64 FROM employee ORDER BY cosineSimilarity DESC LIMIT 1'
    )

    if not query_result.empty:
        result_employee_id, result_employee_name, cosine_similarity, image_base64 = query_result.iloc[0]
        if cosine_similarity > threshold:
            display_attendance_info(frame, result_employee_id, result_employee_name, cosine_similarity, facial_area)
            status_kehadiran = cek_kehadiran(result_employee_id, result_employee_name)
            if len(status_kehadiran) == 0:
                record_attendance(result_employee_id, result_employee_name)
                update_or_insert_employee(result_employee_id, result_employee_name, embedding, image_base64)
        else:
            info_text = 'Wajah tidak dikenali'
            x, y, w, h = facial_area[0], facial_area[1], facial_area[2], facial_area[3]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            cv2.putText(frame, info_text, (x - 30, y + h + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow('Capture Face', frame)
            #print("Wajah tidak ditemukan. Coba lagi!")
    else:
        print("Wajah tidak ditemukan. Coba lagi!")

def main():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        cv2.imshow('Capture Face', frame)

        try:
            face_representation, base64_representation, facial_area = get_face_representation(frame)
            compare_faces(frame, face_representation, facial_area)

        except Exception as e:
            #print(f'Error: {e}')
            pass
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    client = Client(host='localhost', port=9000, user='default', database='default')
    threshold = 0.825

    main()